# Online and Batch Predictions using the Gradio API

In [ ]:
# Install the gradio_client package silently
!pip install -q gradio_client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00


In [ ]:
import time  # Import the time module for time-related functions
from gradio_client import Client  # Import the Gradio client for interacting with Gradio interfaces
from sklearn.datasets import fetch_openml  # Import fetch_openml to fetch datasets
from sklearn.model_selection import train_test_split  # Import train_test_split for splitting datasets
from tqdm import tqdm  # Import tqdm for progress bars

# Test Data

In [ ]:
import pandas as pd  # Import pandas for data manipulation
from sklearn.model_selection import train_test_split  # Import train_test_split for splitting datasets

# Read the dataset from the CSV file into a DataFrame
data_df = pd.read_csv("Bank_Telemarketing.csv")

# Drop the specified columns from the DataFrame
data_df = data_df.drop(columns=['customer_id', 'email_id', 'first_name', 'last_name'])

# Define numerical and categorical feature columns
numerical_features = data_df[['Age', 'Duration(Sec)', 'CC Contact Freq', 'Days Since PC', 'PC Contact Freq']].columns
categorical_features = data_df.select_dtypes(include=['object']).columns

# Separate features (X) and target variable (y)
X = data_df.drop('subscribed', axis=1)
y = data_df['subscribed']

# Split the dataset into training and testing sets
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

In [ ]:
# Check the shape of the testing features dataset
Xtest.shape

(8237, 15)

In [ ]:
# Display the first 3 rows of the testing features dataset
Xtest.head(3)

,Age,Job,Marital Status,Education,Defaulter,Home Loan,Personal Loan,Communication Type,Last Contacted,Day of Week,Duration(Sec),CC Contact Freq,Days Since PC,PC Contact Freq,PC Outcome
28870,45.0,blue-collar,married,Others,no,yes,no,cellular,apr,thu,944,1,0,1,failure
24606,46.0,entrepreneur,married,university degree,no,yes,yes,cellular,nov,mon,78,2,0,1,failure
5382,26.0,services,single,high school,no,no,no,telephone,may,fri,211,4,0,0,nonexistent


In [ ]:
# Rearrange the columns in to numerical and categorical features
Xtest = Xtest[['Age', 'Duration(Sec)', 'CC Contact Freq', 'Days Since PC',
               'PC Contact Freq','Job', 'Marital Status', 'Education', 'Defaulter', 'Home Loan',
               'Personal Loan', 'Communication Type', 'Last Contacted', 'Day of Week',
               'PC Outcome']]

In [ ]:
# Retrieve the values of a specific row (index 5382) from the testing features dataset and convert them to a list
Xtest.loc[5382].tolist()

[26.0,
 211,
 4,
 0,
 0,
 'services',
 'single',
 'high school',
 'no',
 'no',
 'no',
 'telephone',
 'may',
 'fri',
 'nonexistent']

In [ ]:
# Randomly sample 100 rows from the testing features dataset
Xtest_sample = Xtest.sample(100)

In [ ]:
# Convert the sampled rows from the DataFrame to a list of tuples
Xtest_sample_rows = list(Xtest_sample.itertuples(index=False, name=None))

In [ ]:
# Access the first tuple in the list of sampled rows
Xtest_sample_rows[0]

(41.0,
 608,
 2,
 0,
 0,
 'blue-collar',
 'married',
 'experience',
 'no',
 'yes',
 'yes',
 'cellular',
 'apr',
 'tue',
 'nonexistent')

# Inference

In [1]:
# Create a Gradio client instance for the specified Gradio interface
client = Client("---------PASTE YOUR GRADIO API----------")

## Online/Real time

In [ ]:
# Submit a prediction request to the Gradio interface with the given input values
job = client.predict(
    age=3,
    duration=3,
    cc_contact_freq=3,
    days_since_pc=3,
    pc_contact_freq=3,
    job="admin.",
    marital_status="married",
    education="experience",
    defaulter="yes",
    home_loan="yes",
    personal_loan="yes",
    communication_type="cellular",
    last_contacted="may",
    day_of_week="thu",
    pc_outcome="success",
    api_name="/predict"
)

In [ ]:
# Print the prediction result returned by the Gradio interface
print(job)

{'label': '0', 'confidences': None}


## Batch

In [ ]:
# Initialize an empty list to store batch predictions
batch_predictions = []

In [ ]:
from tqdm import tqdm  # Import tqdm for progress bars
import time  # Import time for time-related functions

batch_predictions = []  # Initialize an empty list to store batch predictions

# Iterate through the sampled rows and submit prediction requests to the Gradio interface
for row in tqdm(Xtest_sample_rows):
    try:
        # Submit a prediction request for the current row
        job = client.submit(
            age=row[0],
            duration=row[1],
            cc_contact_freq=row[2],
            days_since_pc=row[3],
            pc_contact_freq=row[4],
            job=row[5],
            marital_status=row[6],
            education=row[7],
            defaulter=row[8],
            home_loan=row[9],
            personal_loan=row[10],
            communication_type=row[11],
            last_contacted=row[12],
            day_of_week=row[13],
            pc_outcome=row[14],
            api_name="/predict"
        )

        # Wait for the job to complete and get the result
        job_result = job.result()

        # Append the prediction result to batch_predictions
        batch_predictions.append(job_result)

        # Sleep for 1 second before processing the next row
        time.sleep(1)

    except Exception as e:
        # Print any exceptions that occur during prediction
        print(f"Error processing row {row}: {e}")
        # If you want to continue processing other rows even after an error,
        # comment out the following line. Otherwise, the loop will stop on the first error.
        # break

100%|██████████| 100/100 [02:02<00:00,  1.23s/it]


In [ ]:
# Display the first 14 predictions from the batch_predictions list
batch_predictions[0:14]

[{'label': '0', 'confidences': None},
 {'label': '0', 'confidences': None},
 {'label': '0', 'confidences': None},
 {'label': '0', 'confidences': None},
 {'label': '0', 'confidences': None},
 {'label': '0', 'confidences': None},
 {'label': '0', 'confidences': None},
 {'label': '0', 'confidences': None},
 {'label': '0', 'confidences': None},
 {'label': '0', 'confidences': None},
 {'label': '0', 'confidences': None},
 {'label': '0', 'confidences': None},
 {'label': '0', 'confidences': None},
 {'label': '0', 'confidences': None}]